# Reassign ID in AO
This notebook remap ID in annotation ontology (AO). ID range changes from uint32 to uint16.

- inputs
    - RL_VC.json
    - annotation_100_segmented_combined_gene_fiber_LR.nrrd
- outputs
    - RL_VC_reID.json
    - LR_reID.nrrd
    - IDnameVC_RL.csv

In [15]:
dir_data = 'data'

# forFigs
fn_input_AO = '1_VC.json'
fn_output_AO = '1_VC_remapID.json'
fn_output_csv = 'remapIDpairs_base.csv'

In [14]:
import os
import pandas as pd
import json
import copy
from collections import OrderedDict
from jsonpath_rw import jsonpath, parse

# Load data

In [16]:
with open(os.path.join(dir_data, fn_input_AO)) as f:
    df_AO = json.load(f, object_pairs_hook=OrderedDict)

# Reassign ID in AO

In [17]:
def reassign_ID(match_id, match_fullpath): 
    global df_AOnew
    global temp_ID
    old_ID = match_id
    new_ID = temp_ID
    temp_ID +=1
    exec("df_AOnew['msg'][0]"+\
         str(match_fullpath).replace('.','')\
          .replace('children', "['children']").replace('id', "['id']")+\
         "= "+ str(new_ID))
    exec("voxel_count = df_AOnew['msg'][0]"+\
         str(match_fullpath).replace('.','').\
         replace('children', "['children']").\
         replace('id', "")+"['voxel_count']", globals())
    exec("acronym = df_AOnew['msg'][0]"+\
         str(match_fullpath).replace('.','').\
         replace('children', "['children']").\
         replace('id', "")+"['acronym']", globals())
    exec("name = df_AOnew['msg'][0]"+\
         str(match_fullpath).replace('.','').\
         replace('children', "['children']").\
         replace('id', "")+"['name']", globals())
    exec("color_hex = df_AOnew['msg'][0]"+\
         str(match_fullpath).replace('.','').\
         replace('children', "['children']").\
         replace('id', "")+"['color_hex_triplet']", globals())
    parent_id = eval("df_AOnew['msg'][0]"+\
                    str(match_fullpath)[0:str(match_fullpath).rfind(".children")+1]\
                    .replace('.','').replace('children',"['children']")+"['id']")
    if match_id == 4000000000:# manually assigned root id
        parent_id = None
    exec("df_AOnew['msg'][0]"+\
         str(match_fullpath).replace('.','')\
          .replace('children', "['children']").replace('id', "['parent_structure_id']")+\
         "= "+ str(parent_id))
    return [old_ID, new_ID, acronym, name, voxel_count, color_hex]

In [18]:
jsonpath_expr = parse('$..id')
df_AOnew = copy.deepcopy(df_AO)
temp_ID = 1

oldnewID_list = [reassign_ID(match.value, match.full_path) \
                 for match in jsonpath_expr.find(df_AO['msg'][0])]
# oldnewID = pd.DataFrame(oldnewID_list, columns = ['old_ID', 'new_ID', 'voxel_count'])
oldnewID = pd.DataFrame(oldnewID_list, columns =\
                        ['old_ID', 'new_ID', 'acronym', 'name', 'voxel_count', 'color'])

# Save AO with reassigned ID

In [19]:
with open(os.path.join(dir_data, fn_output_AO), mode='w') as fw:
    json.dump(df_AOnew, fw, indent=4)

oldnewID.to_csv(os.path.join(dir_data, fn_output_csv), index=False)

# Check data

In [20]:
oldnewID.info() # 322 non-null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1327 entries, 0 to 1326
Data columns (total 6 columns):
old_ID         1327 non-null int64
new_ID         1327 non-null int64
acronym        1327 non-null object
name           1327 non-null object
voxel_count    669 non-null float64
color          1327 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 62.3+ KB


In [21]:
oldnewID

,old_ID,new_ID,acronym,name,voxel_count,color
0,997,1,root,root,3589.0,FFFFFF
1,8,2,grey,Basic cell groups and regions,NaN,BFDAE3
2,567,3,CH,Cerebrum,NaN,B0F0FF
3,688,4,CTX,Cerebral cortex,NaN,B0FFB8
4,695,5,CTXpl,Cortical plate,NaN,70FF70
5,315,6,Isocortex,Isocortex,NaN,70FF71
6,184,7,FRP,"Frontal pole, cerebral cortex",NaN,268F45
7,68,8,FRP1,"Frontal pole, layer 1",237.0,268F45
8,667,9,FRP2/3,"Frontal pole, layer 2/3",245.0,268F45
9,526157192,10,FRP5,"Frontal pole, layer 5",371.0,268F45
